### **Code : Making result tables by year/month**
- Writer : Donghyeon Kim
- Date : 2022.09.13.
- 대상 : 통합 데이터(태양광 사용 & 미사용)

<hr>

### **Prior Settings**

In [1]:
# 라이브러리 설정
import os
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 사용자 함수 호출
# get_project_root : 파일의 상위-상위 경로 호출
# get_name_root : 모든 사용자 이름 호출
# get_name_root_use : 태양광 사용자 이름 호출
# get_name_root_not : 태양광 미사용자 이름 호출
from pack_utils import get_project_root, get_name_root, get_name_root_use, get_name_root_not, kw_dict, kw_value_dict

### **Data**
#### 1) Hour(1시간 단위)

In [3]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_merge_wt_f')
file_name = os.path.join(folder_root, 'all_concat_hour.xlsx')
df_all_hour = pd.read_excel(file_name)

# Filtering
# (1) Using Solar
df_all_hour_use = df_all_hour[df_all_hour.type == 'use']

# (2) Not Using Solar
df_all_hour_not = df_all_hour[df_all_hour.type == 'not']

In [4]:
df_all_hour_use.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97547 entries, 0 to 97546
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   가구번호                    97547 non-null  int64  
 1   연도                      97547 non-null  int64  
 2   월                       97547 non-null  int64  
 3   일                       97547 non-null  int64  
 4   시간                      97547 non-null  int64  
 5   설비용량(kW)                97547 non-null  object 
 6   발전량(kWh)                97547 non-null  float64
 7   전력소비량(kWh)              97547 non-null  float64
 8   수전전력량(kWh)              97547 non-null  float64
 9   잉여전력량(kWh)              97547 non-null  float64
 10  잉여전력량/발전량               47471 non-null  float64
 11  자가소비율                   97546 non-null  float64
 12  자가공급률                   97547 non-null  float64
 13  temperature             97547 non-null  float64
 14  uws_10m                 97547 non-null

#### 2) Day(1일 단위)

In [5]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_merge_wt_f')
file_name2 = os.path.join(folder_root, 'all_concat_day.xlsx')
df_all_day = pd.read_excel(file_name2)

# Filtering
# (1) Using Solar
df_all_day_use = df_all_day[df_all_day.type == 'use']

# (2) Not Using Solar
df_all_day_not = df_all_day[df_all_day.type == 'not']

In [7]:
df_all_day_use.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4150 entries, 0 to 4149
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   가구번호        4150 non-null   int64  
 1   연도          4150 non-null   int64  
 2   월           4150 non-null   int64  
 3   일           4150 non-null   int64  
 4   설비용량(kW)    4150 non-null   object 
 5   발전량(kWh)    4150 non-null   float64
 6   발전시간        4150 non-null   float64
 7   이용률         4150 non-null   float64
 8   전력소비량(kWh)  4150 non-null   float64
 9   수전전력량(kWh)  4150 non-null   float64
 10  잉여전력량(kWh)  4150 non-null   float64
 11  잉여전력량/발전량   4146 non-null   float64
 12  자가소비율       4146 non-null   float64
 13  자가공급률       4150 non-null   float64
 14  type        4150 non-null   object 
 15  owner       4150 non-null   object 
 16  ym          4150 non-null   object 
dtypes: float64(9), int64(4), object(4)
memory usage: 583.6+ KB


<hr>

### **1. 전력 소비량**
사용/미사용 가구 - 월별 평균 전력 소비량

In [8]:
# Result Dictionary
result = {}
result['설치가구(A)'] = []
result['미설치가구(B)'] = []
result['A/B'] = []

# Fill Dictionary
date_list = ['2021/4', '2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3']
for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    df_all_day_not_f = df_all_day_not[df_all_day_not.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['전력소비량(kWh)'])
    not_consum = np.sum(df_all_day_not_f['전력소비량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    not_consum_mean = round(not_consum / len(df_all_day_not_f.owner.unique()), 1)
    calc = round(use_consum_mean / not_consum_mean, 2)
    
    # 4) 값 넣기
    result['설치가구(A)'].append(use_consum_mean)
    result['미설치가구(B)'].append(not_consum_mean)
    result['A/B'].append(calc)
    
result = pd.DataFrame(result)
result.index = date_list
result

,설치가구(A),미설치가구(B),A/B
2021/4,334.8,231.7,1.44
2021/5,339.7,250.0,1.36
2021/6,327.3,263.1,1.24
2021/7,476.9,364.8,1.31
2021/8,453.8,338.1,1.34
2021/9,351.7,272.4,1.29
2021/10,364.9,250.3,1.46
2021/11,381.0,265.8,1.43
2021/12,417.1,293.6,1.42
2022/1,481.0,304.5,1.58


<hr>

### **2. 자가 발전량**
사용 가구(3kW) - 월별 발전량(전력 생산량)

In [10]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []
result['발전시간(h)'] = []
result['이용률(%)'] = []

# Fill Dictionary
date_list = ['2021/4', '2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3']
for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_3kw = df_all_day_use[df_all_day_use['설비용량(kW)'] == '3kW']
    df_all_day_use_f = df_all_day_use_3kw[df_all_day_use_3kw.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['발전량(kWh)'])
    use_hour = np.sum(df_all_day_use_f['발전시간'])
    use_rate = np.sum(df_all_day_use_f['이용률'])

    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    use_hour_mean = round(use_hour / len(df_all_day_use_f), 1)
    use_rate_mean = round(use_rate / len(df_all_day_use_f), 1)

    # 4) 값 넣기
    result['발전량(kWh)'].append(use_consum_mean)
    result['발전시간(h)'].append(use_hour_mean)
    result['이용률(%)'].append(use_rate_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,발전량(kWh),발전시간(h),이용률(%)
2021/4,403.1,4.5,18.7
2021/5,370.8,4.0,16.6
2021/6,363.2,4.0,16.8
2021/7,360.1,3.9,16.1
2021/8,287.6,3.1,12.9
2021/9,291.4,3.2,13.5
2021/10,320.9,3.5,14.4
2021/11,259.8,2.9,12.0
2021/12,256.0,2.8,11.5
2022/1,290.4,3.1,13.0


<hr>

### **3. 잉여 전력량**
사용 가구 - 월별 평균 잉여 전력량

In [12]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []
result['잉여전력량(kWh)'] = []
result['잉여전력량/발전량'] = []

# Fill Dictionary
date_list = ['2021/4', '2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3']
for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['발전량(kWh)'])
    use_export = np.sum(df_all_day_use_f['잉여전력량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    use_export_mean = round(use_export / len(df_all_day_use_f.owner.unique()), 1)
    calc = round(use_export_mean / use_consum_mean, 2)
    
    # 4) 값 넣기
    result['발전량(kWh)'].append(use_consum_mean)
    result['잉여전력량(kWh)'].append(use_export_mean)
    result['잉여전력량/발전량'].append(calc)
    
result = pd.DataFrame(result)
result.index = date_list
result

,발전량(kWh),잉여전력량(kWh),잉여전력량/발전량
2021/4,531.0,416.1,0.78
2021/5,489.8,376.4,0.77
2021/6,475.0,353.0,0.74
2021/7,473.0,315.2,0.67
2021/8,387.1,256.1,0.66
2021/9,367.0,260.9,0.71
2021/10,422.1,319.6,0.76
2021/11,348.2,260.7,0.75
2021/12,351.4,256.5,0.73
2022/1,391.2,275.3,0.70


<hr>

### **4. 수전 전력량**
사용/미사용 가구 - 월별 평균 수전 전력량

In [14]:
# Result Dictionary
result = {}
result['설치가구(A)'] = []
result['미설치가구(B)'] = []
result['A/B'] = []

# Fill Dictionary
date_list = ['2021/4', '2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3']
for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    df_all_day_not_f = df_all_day_not[df_all_day_not.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['수전전력량(kWh)'])
    not_consum = np.sum(df_all_day_not_f['수전전력량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    not_consum_mean = round(not_consum / len(df_all_day_not_f.owner.unique()), 1)
    calc = round(use_consum_mean / not_consum_mean, 2)
    
    # 4) 값 넣기
    result['설치가구(A)'].append(use_consum_mean)
    result['미설치가구(B)'].append(not_consum_mean)
    result['A/B'].append(calc)
    
result = pd.DataFrame(result)
result.index = date_list
result

,설치가구(A),미설치가구(B),A/B
2021/4,219.9,231.7,0.95
2021/5,226.3,250.0,0.91
2021/6,205.3,263.1,0.78
2021/7,319.1,364.8,0.87
2021/8,322.8,338.1,0.95
2021/9,245.6,272.4,0.90
2021/10,262.3,250.3,1.05
2021/11,293.4,265.8,1.10
2021/12,322.2,293.6,1.10
2022/1,365.0,304.5,1.20


<hr>

### **5. 자가소비율(SCR)**
사용 가구 - 월별 자가소비율

In [15]:
# Result Dictionary
result = {}
result['300W'] = []
result['3kW'] = []
result['18kW'] = []

# Fill Dictionary
date_list = ['2021/4', '2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3']
for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 설비용량 구분
    df_300w = df_all_day_use_f[df_all_day_use_f['설비용량(kW)'] == '300W']
    df_3kw = df_all_day_use_f[df_all_day_use_f['설비용량(kW)'] == '3kW']
    df_18kw = df_all_day_use_f[df_all_day_use_f['설비용량(kW)'] == '18kW']
    
    # 3) 발전용량별 자가소비율 합산
    scr_300w = np.sum(df_300w['자가소비율']) / len(df_300w)
    scr_3kw = np.sum(df_3kw['자가소비율']) / len(df_3kw)
    scr_18kw = np.sum(df_18kw['자가소비율']) / len(df_18kw)
    
    # 4) 평균 구하기
    scr_300w_mean = round(scr_300w * 100, 1)
    scr_3kw_mean = round(scr_3kw * 100, 1)
    scr_18kw_mean = round(scr_18kw * 100, 1)
    
    # 5) 값 넣기
    result['300W'].append(scr_300w_mean)
    result['3kW'].append(scr_3kw_mean)
    result['18kW'].append(scr_18kw_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,300W,3kW,18kW
2021/4,97.9,35.4,9.0
2021/5,98.1,40.4,9.9
2021/6,98.9,41.0,8.7
2021/7,96.1,50.1,12.3
2021/8,98.9,54.9,13.3
2021/9,97.2,47.1,11.0
2021/10,98.2,40.7,9.4
2021/11,97.4,41.9,10.9
2021/12,98.3,45.1,8.4
2022/1,96.9,48.4,8.1


<hr>

### 6. **자가공급률(SSR)**
사용 가구 - 월별 자가공급률

In [16]:
# Result Dictionary
result = {}
result['300W'] = []
result['3kW'] = []
result['18kW'] = []

# Fill Dictionary
date_list = ['2021/4', '2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3']
for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 설비용량 구분
    df_300w = df_all_day_use_f[df_all_day_use_f['설비용량(kW)'] == '300W']
    df_3kw = df_all_day_use_f[df_all_day_use_f['설비용량(kW)'] == '3kW']
    df_18kw = df_all_day_use_f[df_all_day_use_f['설비용량(kW)'] == '18kW']
    
    # 3) 발전용량별 자가소비율 합산
    ssr_300w = np.sum(df_300w['자가공급률']) / len(df_300w)
    ssr_3kw = np.sum(df_3kw['자가공급률']) / len(df_3kw)
    ssr_18kw = np.sum(df_18kw['자가공급률']) / len(df_18kw)
    
    # 4) 평균 구하기
    ssr_300w_mean = round(ssr_300w * 100, 1)
    ssr_3kw_mean = round(ssr_3kw * 100, 1)
    ssr_18kw_mean = round(ssr_18kw * 100, 1)
    
    # 5) 값 넣기
    result['300W'].append(ssr_300w_mean)
    result['3kW'].append(ssr_3kw_mean)
    result['18kW'].append(ssr_18kw_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,300W,3kW,18kW
2021/4,9.6,33.7,80.3
2021/5,8.2,34.6,78.9
2021/6,8.3,39.2,79.3
2021/7,5.8,35.6,73.0
2021/8,6.3,30.7,69.4
2021/9,7.4,31.5,74.9
2021/10,5.2,29.4,76.0
2021/11,4.3,23.1,71.0
2021/12,4.5,22.7,70.8
2022/1,4.6,24.0,71.7
